## 2.10. 트레이너

허깅페이스 트레이너(Trainer)는 모델 학습 프로세스 전반을 추상화해 간소화된 인터페이스를 제공한다. 이를 통해 몇 줄의 코드만으로도 복잡한 머신러닝 모델을 효율적으로 학습시킬 수 있다.

**트레이너의 장점**

---
* **코드 재사용성 및 유지보수성 향상**: 트레이너는 잘 테스트되고 검증된 코드를 사용하므로, 직접 구현하는 것보다 안정적이고 재사용 가능한 코드를 작성할 수 있다. 이를 통해 코드의 품질과 일관성을 높일 수 있다.
* **범용성**: 트레이너는 다양한 유형의 모델과 작업에 범용적으로 사용할 수 있다. 텍스트, 이미지, 오디오 등 다양한 데이터 유형을 지원하며, 분류, 회귀, 시퀀스 레이블링 등 다양한 작업을 지원한다.
* **모니터링 및 제어**: 콜백 함수를 사용해 학습 프로세스를 모니터링하고 제어할 수 있다. 특정 이벤트가 발생했을 때 원하는 작동 방식을 저으이할 수 있다. 

허깅페이스의 트레이너는 머신러닝 모델 학습 프로세스를 간소화하고 강력한 기능을 제공한다. 트레이너는 모델, 데이터세트, 최적화 기법, 학습률 스케쥴러 등을 통합적으로 관리해 학습 프로세스 전반에 걸친 설정을 손쉽게 조정할 수 있대요 ....



### 2.10.1. 트레이너 클래스

허깅페이스의 트레이너 클래스는 머신러닝 모델 학습 작업을 단순화하고 체계화했다. 

**트레이너 클래스의 주요 매개변수**

---
* **model**: 학습, 평가 또는 예측에 사용할 모델 인스턴스다 `PreTrainedModel` 또는 `torch.nn.Module` 클래스의 인스턴스여야 한다. 모델을 제공하지 않으면 `model_init`을 전달해야 한다.
* **args**: 학습 관련 설정을 조정하기 위한 트레이닝 인수다. 입력하지 않으면 지맘대로 한다. 출력 디렉토리는 현재 디렉토리에 tmp_trainer 디렉토리를 만들어 저장한다. 정말 지맘대로 한다.
* **data_collator**: 학습 또는 평가 데이터세트에서 추출된 배치를 모델에 입력하기 전에 패딩, 마스킹 등의 작업을 수행하기 위한 함수를 입력한다.
* **train_dataset**: 학습에 사용할 데이터세트다. 모델의 순방향 메서드에 허용하지 않는 열은 자동으로 제거된다. 분산 환경에서 학습할 경우, 데이터세트는 모든 프로세스에서 동일한 시드를 사용하거나 `set_epoch` 메서드를 구현해야 한다.
* **eval_dataset**: 평가에 사용할 데이터세트다. 딕셔너리인 경우 각 데이터세트에 대해 키를 지표 이름 앞에 추가해 평가한다.
* **tokenizer**: 데이터 전처리에 사용할 `PreTrainedTokenizer`인스턴스다. 배치 생성 시 자동으로 입력을 최대 길이로 패딩한다. 토크나이저는 모델과 함께 저장되어 학습이 중단 돼더라도 재사용 할 수 있다.
* **model_init**: 모델 인스턴스를 생성하는 함수를 설정한다. `train`메서드가 호출될 때마다 새로운 모델 인스턴스가 생성된다.
* **compute_metrics**: 평가 시 지표를 계산하는 함수를 입력한다. 정확도나 F1 점수 등을 이용해 모델을 평가한다.
* **callbacks**: 학습 프로세스 중에 특정 이벤트가 발생했을 때 실행될 사용자 정의 함수를 등록한다. 에폭이 끝날 때마다 모델 성능을 평가하고 싶을 때나 학습 중 특정 조건에 따라 조기 종료하고 싶을 때 사용한다.
* **optimizers**: 최적화 알고리즘을 설정한다. `AdamW`, `Adafactor`, `Adamax` 등을 사용할 수 있다. 기본은 `AdamW`다.역시 지맘대로 한다.
* **preprocess_logits_for_metrics**: 지표 계산을 위해 모델 출력을 전처리하는 함수를 입력한다. 기본값은 `None`이다.
* 

### 2.10.2. 트레이닝 인수

트레이닝 인수는 학습 하이퍼파라미터와 설정을 캡슐화한 클래스다. 트레이너 클래스에 전달되어 학습 프로세스를 제어하는데 사용된다. 이렇게 목적을 분리해 특정 모델이나 과제에 구애받지 않고 재사용 가능한 인터페이스로 구축할 수 있다.

```python
training_args = transformers.TrainingArguments(...)
```
* **output_dir**: 모델 예측 결과와 체크포인트가 저장될 출력 경로를 지정한다.
* **overwrite_output_dir**: 출력 경로에 이미 파일이 있을 때 덮어쓸지 여부를 지정한다. 이전에 학습을 중단했던 체크포인트 디렉토리에서 학습을 재개하고 싶은 경우에도 `True`로 설정 할때가 있댄다. 기본값은 `False`다.
* **do_train**: 학습을 수행할지 여부를 지정한다. `False`로 설정하면 학습을 수행하지 않는다. 기본값은 `False`다.
* **do_eval**: 모델 학습중 검증 데이터세트에 대한 평가 수행 여부를 설정한다. eval_strategy가 no가 아닌경우 True로 설정된다. 기본값은 `False`다.
* **do_predict**: 테스트 데이터세트에 대한 예측을 수행할지 여부를 지정한다. 일반적으로 모델 평가에서 사용된다. 기본값은 `False`다.
* **eval_strategy**: 모델 학습 중 평가 전략을 설정한다. `no`로 설정하면 평가를 수행하지 않는다. steps로 설정한 경우 eval_steps 간격마다 평가를 수행하고 로그를 기록한다. epoch로 설정하는 경우 각 에폭 종료 후 평가를 수행한다.
* **per_device_train_batch_size**: 학습 중 각 장치에 할당된 배치 크기를 설정한다. 기본값은 8이다. GPU가 2개 이면 16으로 설정되는 것이다.
* **per_device_batch_size**: 모델 평가 시 장치당 배치 크기를 의미한다. per_device_train_batch_size와 동일하게 설정한다.
* **eval_delay**: 첫 평가를 수행하기 전에 기다려야하느 에폭 또는 스텝 수를 설정한다.
* **learning_rate**: 학습률을 설정한다. 기본값은 5e-5다.
* **weight_decay**: 가중치 감쇠 값을 설정한다. 기본값은 0.0이다.
* **adam_beta1, adam_beta2, adam_epsilon**: Adam 옵티마이저의 베타1, 베타2, 엡실론 값을 설정한다. 기본값은 0.9, 0.999, 1e-8이다.
* **max_grad_norm**: 그레이디언트 클리핑을 위한 최대 그레이디언트 노름 값을 설정한다. 기본값은 1.0이다.
* **num_train_epochs**: 수행할 총 에폭 수를 설정한다. 기본값은 3이다.
* **max_steps**: 최대 학습 스텝 수를 설정한다. 양의 정수로 설정하면 num_train_epochs를 무시하고 해당 스텝까지만 학습한다. 기본값은 -1이다.
* **warmup_steps**: 학습률 스케쥴러에서 웜업 스텝 수를 설정한다. 학습 초기에 학습률을 점진적으로 증가시키는 기법으로 웜업 단계동안 한습률은 일정 스케쥴에 따라 점진적으로 증가한다. 기본값은 0이다.
* **logging_dir**: 텐서보드 로그를 저장할 디렉토리 경로를 지정한다. 기본값은 `runs`다.
* **logging_strategy**: 모델 학습에 사용할 로깅 전략으로 no, epoch, steps를 지정할 수 있다. 기본값은 steps다.
* **logging_steps**: 로깅 간격을 설정한다. 기본값은 500이다.
* **save_strategy**: 모델 체크포인트 저장 전략을 설정한다. no, epoch, steps를 지정할 수 있다. 기본값은 steps다.
* **save_steps**: 체크포인트 저장 간격을 설정한다. `save_strategy`가 `steps`로 설정된 경우에 체크포인트를 저장할 업데이트 스텝 간격을 의미한다. 0과 1사이에 실수를 입력하면 총 학습 스텝의 비율로 지정된다.
* **save_total_limit**: 저장할 체크포인트의 최대 개수를 설정한다. 기본값은 5다. 지정수를 초과하면 가장 오래된 체크포인트를 삭제한다. `load_best_model_at_end`가 `True`로 설정된 경우 최고 성능의 모델은 삭제되지 않는다.
* **seed**: 랜덤 시드를 설정한다. 학습을 재현하기 위해 사용한다. 기본값은 42다.
* **fp16**: 16비트 혼합 정밀도 학습 수행 여부를 설정한다. True로 설정하면 32비트 대신 16비트 혼합 정밀도로 학습을 수행한다.
* **fp16_opt_level**: pf16 학습 시 Apex AMP(Nvidia에서 제공하는 자동 혼합 정밀도)최적화 수준을 설정한다. 01, 02등을 지정할 수 있다.
* **run_name**: WandB, MLflow등의 로깅 도구에서 사용할 레이블을 설정한다. 설정하지 않으면 output_dir과 동일하게 할당된다.
* **load_best_model_at_end**: 학습 종료 후 가장 성능이 좋았던 모델을 불러올지 여부를 설정한다. True를 설정하면 최적 체크포인트가 항상 보존된다.
* **metric_for_best_model**: 평가지표에서 큰 값이 더 좋은 모델인지 여부를 지정한다. 예를 들어 `metric_for_best_model=eval_accuracy`로 지정했다면 `greater_is_better=True`로 해야 정확도가 높은 모델이 최적 모델로 선택되며, 반대로 `metric_for_best_model="eval_loss"`로 지정했다면 `greater_is_better=False`로 설정해야 손실이 낮은 모델이 최적 모델로 선택된다.
* **optim**: 최적화 함수를 설정한다. `AdamW`, `Adafactor`, `Adamax` 등을 사용할 수 있다. 기본은 `AdamW`다.
* **optim_args**: 최적화 함수에 전달할 추가 인수를 할당한다.
* **report_to**: 결과와 로그를 전달할 플랫폼을 설정한다. wandb, tensorboard, mlflow 등을 지정할 수 있다.
* **push_to_hub**: 모델이 저장될 때마다 허깅페이스 허브에 모델을 업로드할지 설정한다.
* **resume_from_checkpoint**: 이전에 저장한 체크포인트에서 학습을 재개할지 여부를 설정한다. 경로를 지정하면 해당 체크포인트에서 학습을 재개한다.

### 2.10.3. 토큰 분류 - 개체명 인식

토큰 분류(Token classification)모델은 자연어 처리 분야에서 문장을 구성하는 각 토큰을 할당하는 모델을 의미한다. 개체명 인식 모델은 문장에서 사람, 조직, 위치 등의 개체명을 인식하고 해당 개체명에 대한 범주를 레이블로 할당한다.

**한국정보통신기술협회에서 표준으로 제시한 개체명 인식 레이블**

... ~ 어디 있는건지 출처가 .....


**KLUE 데이터세트를 이용한 개채명 인식**

KLUE(Korean Language Understanding Evaluation) 데이터세트는 한국어 자연어 처리 모델을 평가하기 위한 데이터세트다. KLUE 데이터세트는 다양한 자연어 처리 작업을 위한 데이터세트로 구성돼 있다. 이 데이터세트를 이용해서 개채명 인식을 한댄다. load_dataset을 이용해 데이터세트를 불러온다. ner는 (Named Entity Recognition)개체명 인식을 의미한다. 예제 코드에도 객체명 인식을 위한 모델이 사용된다. 즉 목적에 맞는 데이터세트가 있고 그 데이터세트를 이용하는 모델을 사용했다.

In [1]:
# KLUE 데이터세트 및 개체명 인식 모델 초기화

from datasets import load_dataset
from transformers import AutoModelForTokenClassification, AutoTokenizer

# KLUE 데이터세트 
dataset = load_dataset("klue", "ner")
print(dataset)
labels = dataset["train"].features["ner_tags"].feature.names

model_name = "Leo97/KoELECTRA-small-v3-modu-ner"

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForTokenClassification.from_pretrained(
  model_name,
  num_labels=len(labels),
  ignore_mismatched_sizes=True
)

print(dataset["train"])
print(dataset["train"][0])
print(labels)

Reusing dataset klue (/home/asanobm/.cache/huggingface/datasets/klue/ner/1.0.0/e0fc3bc3de3eb03be2c92d72fd04a60ecc71903f821619cb28ca0e1e29e4233e)


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['sentence', 'tokens', 'ner_tags'],
        num_rows: 21008
    })
    validation: Dataset({
        features: ['sentence', 'tokens', 'ner_tags'],
        num_rows: 5000
    })
})


Some weights of ElectraForTokenClassification were not initialized from the model checkpoint at Leo97/KoELECTRA-small-v3-modu-ner and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([31]) in the checkpoint and torch.Size([13]) in the model instantiated
- classifier.weight: found shape torch.Size([31, 256]) in the checkpoint and torch.Size([13, 256]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Dataset({
    features: ['sentence', 'tokens', 'ner_tags'],
    num_rows: 21008
})
{'sentence': '특히 <영동고속도로:LC> <강릉:LC> 방향 <문막휴게소:LC>에서 <만종분기점:LC>까지 <5㎞:QT> 구간에는 승용차 전용 임시 갓길차로제를 운영하기로 했다.', 'tokens': ['특', '히', ' ', '영', '동', '고', '속', '도', '로', ' ', '강', '릉', ' ', '방', '향', ' ', '문', '막', '휴', '게', '소', '에', '서', ' ', '만', '종', '분', '기', '점', '까', '지', ' ', '5', '㎞', ' ', '구', '간', '에', '는', ' ', '승', '용', '차', ' ', '전', '용', ' ', '임', '시', ' ', '갓', '길', '차', '로', '제', '를', ' ', '운', '영', '하', '기', '로', ' ', '했', '다', '.'], 'ner_tags': [12, 12, 12, 2, 3, 3, 3, 3, 3, 12, 2, 3, 12, 12, 12, 12, 2, 3, 3, 3, 3, 12, 12, 12, 2, 3, 3, 3, 3, 12, 12, 12, 8, 9, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12]}
['B-DT', 'I-DT', 'B-LC', 'I-LC', 'B-OG', 'I-OG', 'B-PS', 'I-PS', 'B-QT', 'I-QT', 'B-TI', 'I-TI', 'O']


**개체명 인식을 위한 데이터 전처리**

preprocess_data 함수는 입력 데이터를 토크나이저에 적합한 형태로 변환한다. example["token"]에서 "".join을 통해 문자열을 이어 붙이고 `\xa0`문자를 공백문자로 변경한다. 이렇게 하면 줄바꿈 없는 공백(No-Break Space)문자를 공백문자로 변경할 수 있다. 이후 토크나이저를 이용해 입력 데이터를 토큰화한다. 이때 토큰화된 데이터는 토큰과 레이블로 구성돼 있다. tokenizer함수를 사용해 해당 문자열을 토큰화하고, 각 토큰의 시작/끝 위치 정보를 offset_mapping으로 받는다. 토큰화된 결과에 대해 labels를 생성한다. 각 토큰의 offset값을 확인해, 시작과 끝 위치가 같다면 -100(패딩토큰)을 할당한다. 

In [2]:
from datasets import load_dataset
from transformers import AutoModelForTokenClassification, AutoTokenizer


dataset = load_dataset("klue", "ner")

model_name = "Leo97/KoELECTRA-small-v3-modu-ner"
model = AutoModelForTokenClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

def preprocess_data(example, tokenizer):
  sentence = "".join(example["tokens"]).replace("\xa0", " ")
  encoded = tokenizer(
    sentence,
    return_offsets_mapping=True,
    add_special_tokens=False,
    padding=False,
    truncation=False
  )

  labels = []

  for offset in encoded.offset_mapping:
    if offset[0] == offset[1]:
      labels.append(-100)
    else:
      labels.append(example["ner_tags"][offset[0]])
  
  encoded["labels"] = labels
  return encoded


processed_dataset = dataset.map(
  lambda example: preprocess_data(example, tokenizer),
  batched=False,
  remove_columns=dataset["train"].column_names
)

print(processed_dataset)
print(dataset["train"][0]["ner_tags"])
print(processed_dataset["train"][0]["offset_mapping"])
print(processed_dataset["train"][0]["labels"])

Reusing dataset klue (/home/asanobm/.cache/huggingface/datasets/klue/ner/1.0.0/e0fc3bc3de3eb03be2c92d72fd04a60ecc71903f821619cb28ca0e1e29e4233e)


  0%|          | 0/2 [00:00<?, ?it/s]

Parameter 'function'=<function <lambda> at 0x7fa0eabfa3e0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/21008 [00:00<?, ?ex/s]

  0%|          | 0/5000 [00:00<?, ?ex/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'offset_mapping', 'labels'],
        num_rows: 21008
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'offset_mapping', 'labels'],
        num_rows: 5000
    })
})
[12, 12, 12, 2, 3, 3, 3, 3, 3, 12, 2, 3, 12, 12, 12, 12, 2, 3, 3, 3, 3, 12, 12, 12, 2, 3, 3, 3, 3, 12, 12, 12, 8, 9, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12]
[[0, 2], [3, 9], [10, 12], [13, 15], [16, 17], [17, 18], [18, 19], [19, 20], [20, 21], [21, 22], [22, 23], [24, 25], [25, 26], [26, 27], [27, 28], [28, 29], [29, 30], [30, 31], [32, 33], [33, 34], [35, 37], [37, 38], [38, 39], [40, 43], [44, 46], [47, 49], [50, 51], [51, 52], [52, 53], [53, 54], [54, 55], [55, 56], [57, 59], [59, 60], [60, 61], [61, 62], [63, 64], [64, 65], [65, 66]]
[12, 2, 2, 12, 2, 3, 3, 3, 3, 12, 12, 2, 3, 3, 3,

**개체명 인식을 위한 모델 학습**

`TrainingArguments`를 이용해 학습에 필요한 하이퍼파라미터를 설정한다. 여기서는 출력 디렉토리(output_dir), 평가 전략(evlauation_strategy:이거 이제 안쓴대요.), 배치 크기(per_device_train_batch_size, per_device_eval_batch_size), 학습률(learning_rate), 가중치 감소(weight_decay), 에폭 수(num_train_epochs), 시드(seed) 값등의 옵션을 설정했다.

트레이너 객체는 학습시킬 모델, 설정한 `TrainingArguments`객체 학습 데이터세트, 평가 데이터세트, 데이터 콜렉터를 전달 했다. 데이터 콜렉터는 `DatacollatorForTokenClassification` 객체를 사용하고 토크나이저와 패딩 옵션을 추가했다.

학습 과정에서 트레이너는 설정된 하이퍼파라미터와 데이터세트를 사용해 모델 가중치를 갱신한다. 각 에폭마다 평가 데이터세트에 대한 성능 평가를 수행하고, 최적의 모델 가중치를 저장한다.

> 책에는 에폭을 5번 줬다. 궁금해서 한 10번 줘봤다. 겁나게 오래 걸린다 ... ㅠㅠ


In [ ]:
# 개채명 인식 모델 학습
import torch
from transformers import Trainer, TrainingArguments
from transformers.data.data_collator import DataCollatorForTokenClassification

device = "cuda" if torch.cuda.is_available() else "cpu"

model.to(device)

training_args = TrainingArguments(
  output_dir="runs/token-classification",
  evaluation_strategy="epoch",
  per_device_train_batch_size=64,
  per_device_eval_batch_size=64,
  learning_rate=1e-4,
  weight_decay=0.01,
  num_train_epochs=10,
  seed=42,
)

trainer = Trainer(
  model=model,
  args=training_args,
  train_dataset=processed_dataset["train"],
  eval_dataset=processed_dataset["validation"],
  data_collator=DataCollatorForTokenClassification(tokenizer, padding=True),
)

trainer.train()

/home/asanobm/miniforge3/envs/ai/lib/python3.11/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/home/asanobm/miniforge3/envs/ai/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,0.192619
2,No log,0.192404
3,No log,0.205167
4,0.003300,0.204603
5,0.003300,0.201046
6,0.003300,0.205458
7,0.002100,0.209963
8,0.002100,0.210582
9,0.002100,0.209986
10,0.001300,0.211019


/home/asanobm/miniforge3/envs/ai/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/home/asanobm/miniforge3/envs/ai/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/home/asanobm/miniforge3/envs/ai/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/home/asanobm/miniforge3/envs/ai/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


TrainOutput(global_step=1650, training_loss=0.0021054671930544303, metrics={'train_runtime': 75.2039, 'train_samples_per_second': 2793.473, 'train_steps_per_second': 21.94, 'total_flos': 877012588572192.0, 'train_loss': 0.0021054671930544303, 'epoch': 10.0})

**개체명 인식 모델 테스트**

model.eval을 호출해 모델을 평가 모드로 전환한다. 이제 모델의 가중치는 고정되고 드롭아웃과 같은 정규화 기법이 비활성화되도록 한다. 이후 예측할 텍스트를 넣어주면 결과가 나온다.
***그런데 결과를 받기는 정말로 까다로운 코드를 작성해야 한다. 이렬려구 허깅페이스 공부하는게 아닌데 이딴 코드를 작성하게 하는 저자의 저의를 이해할 수 없다.***

**개체명 인식의 한계**

* **데이터 부족**: 학습 데이터에 '위키북스'가 조직으로 레입르링된 예시가 충분하지 않거나 전혀 없을 수 있다. 이런 경우 모델이 '위키북스'를 조직으로 학습하지 못해(?)
* **컨텍스트 부족**: 문장에서'위키북스'가 조직임을 명확히 알 수 있는 맥락이 부족할 수 있다.(이게 정답인거 같기도 하다)
* **모델의 한계**: 모델의 성능이 후달려서 '위키북스'를 조직으로 인식하지 못할 수 있다. 새로운 단어이거나 학습 데이터에서 많이 등장하지 않을 경우 모델이 인식하지 못할 수 있다.
* **서브토큰 처리 문제**: 예시에서 '위키북스'는 '위키'와 '##북', '##스'로 토큰화됐다. 이런 경우 모델이 '위키북스'를 올바르게 인식하지 못할 수 있다.

In [13]:
# 개체명 인식 모델 테스트

import torch

model.eval()

text = """
위키북스의 윤대희, 김동화, 송종민 그리고 진현두는 2025년 서울에서 2시간 동안 신간 1권에 관한 논의를 진행했다.
"""

with torch.no_grad():
  tokenized = tokenizer(
    text,
    return_tensors="pt",
    padding=True,
    truncation=True,
    max_length=512,
    add_special_tokens=False,
  )

  logits = model(**tokenized.to(device)).logits.cpu()

predictions = logits.argmax(dim=-1)[0].tolist()
tokens = tokenizer.tokenize(text)

print(list(zip(tokens, [labels[i] for i in predictions])))

[('위키', 'O'), ('##북', 'O'), ('##스', 'O'), ('##의', 'O'), ('윤', 'B-PS'), ('##대', 'I-PS'), ('##희', 'I-PS'), (',', 'O'), ('김동', 'B-PS'), ('##화', 'I-PS'), (',', 'O'), ('송', 'B-PS'), ('##종', 'I-PS'), ('##민', 'I-PS'), ('그리고', 'O'), ('진', 'B-PS'), ('##현', 'I-PS'), ('##두', 'I-PS'), ('##는', 'O'), ('2025', 'B-DT'), ('##년', 'I-DT'), ('서울', 'B-LC'), ('##에', 'O'), ('##서', 'O'), ('2', 'B-TI'), ('##시간', 'I-TI'), ('동안', 'O'), ('신간', 'O'), ('1', 'B-QT'), ('##권', 'I-QT'), ('##에', 'O'), ('관한', 'O'), ('논의', 'O'), ('##를', 'O'), ('진행', 'O'), ('##했', 'O'), ('##다', 'O'), ('.', 'O')]


그냥 아무거나 넣어 봤다. 결과가 아주 잘 나왔다. 롤링파스타는 개체명으로 인식한게 아니다. 더군다나 연신내 조차 제대로 인식하지 못했다. 안타깞다.

In [ ]:
# 개체명 인식 모델 테스트

import torch

model.eval()

text = """
연신내에 위치 했던 롤링파스타는 폐점했다.
그래서 나는 아직도 단 한번도 롤링파스타를 먹지 못했다.
안타깝지만 가가운 롤링파스타를 먹기 위해서는 멀리까지 가야한다.
하지만 나는 귀찮아서 가지 않을꺼다. 안타까운 현실이다.
지금은 스타벅스에서 코딩질을 하고 있다. 안쓰던 키보드를 썼더니 손목아지가 아작이 났다. 또 파스를 붙혔다. 
오늘의 일기는 여기까지 ㅋㅋㅋㅋ
"""

with torch.no_grad():
  tokenized = tokenizer(
    text,
    return_tensors="pt",
    padding=True,
    truncation=True,
    max_length=512,
    add_special_tokens=False,
  )

  logits = model(**tokenized.to(device)).logits.cpu()

predictions = logits.argmax(dim=-1)[0].tolist()
tokens = tokenizer.tokenize(text)

print(list(zip(tokens, [labels[i] for i in predictions])))

[('연신', 'O'), ('##내', 'O'), ('##에', 'O'), ('위치', 'O'), ('했', 'O'), ('##던', 'O'), ('롤링', 'O'), ('##파', 'O'), ('##스타', 'O'), ('##는', 'O'), ('폐', 'O'), ('##점', 'O'), ('##했', 'O'), ('##다', 'O'), ('.', 'O'), ('그래서', 'O'), ('나', 'O'), ('##는', 'O'), ('아직', 'O'), ('##도', 'O'), ('단', 'O'), ('한', 'B-QT'), ('##번', 'I-QT'), ('##도', 'O'), ('롤링', 'O'), ('##파', 'O'), ('##스타', 'O'), ('##를', 'O'), ('먹', 'O'), ('##지', 'O'), ('못', 'O'), ('##했', 'O'), ('##다', 'O'), ('.', 'O'), ('안타깝', 'O'), ('##지만', 'O'), ('가가', 'O'), ('##운', 'O'), ('롤링', 'O'), ('##파', 'O'), ('##스타', 'O'), ('##를', 'O'), ('먹', 'O'), ('##기', 'O'), ('위해서', 'O'), ('##는', 'O'), ('멀리', 'O'), ('##까', 'O'), ('##지', 'O'), ('가야', 'O'), ('##한다', 'O'), ('.', 'O'), ('하지', 'O'), ('##만', 'O'), ('나', 'O'), ('##는', 'O'), ('귀찮', 'O'), ('##아', 'O'), ('##서', 'O'), ('가지', 'O'), ('않', 'O'), ('##을', 'O'), ('##꺼', 'O'), ('##다', 'O'), ('.', 'O'), ('안타까운', 'O'), ('현실', 'O'), ('##이다', 'O'), ('.', 'O'), ('지금', 'O'), ('##은', 'O'), ('스타', 'B-OG'), ('##벅', 'I-OG'), ('#

그래서 이런 저런 정보를 더 줘 봤다. 연신내는 지명에 해당하지만 연신내역으로 변경하고 은평구까지 넣어주고 맛 없다 라고 "개소리"도 좀더 넣어 줬다.

In [15]:
# 개체명 인식 모델 테스트

import torch

model.eval()

text = """
서울 특별시 은평구의 연신내역 근처에 위치 했던 롤링파스타는 폐점했다.
왜 폐점했는지는 모르지만 아무래도 맛이 없었나보다. 아닌가(?) 나는 안먹어봐서 모른다.
그래서 나는 아직도 단 한번도 롤링파스타를 먹지 못했다.
안타깝지만 가가운 롤링파스타를 먹기 위해서는 멀리까지 가야한다.
하지만 나는 귀찮아서 가지 않을꺼다. 안타까운 현실이다.
지금은 스타벅스에서 코딩질을 하고 있다. 안쓰던 키보드를 썼더니 손목아지가 아작이 났다. 또 파스를 붙혔다. 
오늘의 일기는 여기까지 ㅋㅋㅋㅋ
"""

with torch.no_grad():
  tokenized = tokenizer(
    text,
    return_tensors="pt",
    padding=True,
    truncation=True,
    max_length=512,
    add_special_tokens=False,
  )

  logits = model(**tokenized.to(device)).logits.cpu()

predictions = logits.argmax(dim=-1)[0].tolist()
tokens = tokenizer.tokenize(text)

print(list(zip(tokens, [labels[i] for i in predictions])))

[('서울', 'B-LC'), ('특별시', 'B-LC'), ('은평구', 'I-LC'), ('##의', 'O'), ('연신', 'B-LC'), ('##내', 'I-LC'), ('##역', 'I-LC'), ('근처', 'O'), ('##에', 'O'), ('위치', 'O'), ('했', 'O'), ('##던', 'O'), ('롤링', 'O'), ('##파', 'O'), ('##스타', 'O'), ('##는', 'O'), ('폐', 'O'), ('##점', 'O'), ('##했', 'O'), ('##다', 'O'), ('.', 'O'), ('왜', 'O'), ('폐', 'O'), ('##점', 'O'), ('##했', 'O'), ('##는지', 'O'), ('##는', 'O'), ('모르', 'O'), ('##지만', 'O'), ('아무래도', 'O'), ('맛', 'O'), ('##이', 'O'), ('없', 'O'), ('##었', 'O'), ('##나', 'O'), ('##보', 'O'), ('##다', 'O'), ('.', 'O'), ('아닌가', 'O'), ('(', 'O'), ('?', 'O'), (')', 'O'), ('나', 'O'), ('##는', 'O'), ('안', 'O'), ('##먹', 'O'), ('##어', 'O'), ('##봐', 'O'), ('##서', 'O'), ('모른다', 'O'), ('.', 'O'), ('그래서', 'O'), ('나', 'O'), ('##는', 'O'), ('아직', 'O'), ('##도', 'O'), ('단', 'O'), ('한', 'B-QT'), ('##번', 'I-QT'), ('##도', 'O'), ('롤링', 'O'), ('##파', 'O'), ('##스타', 'O'), ('##를', 'O'), ('먹', 'O'), ('##지', 'O'), ('못', 'O'), ('##했', 'O'), ('##다', 'O'), ('.', 'O'), ('안타깝', 'O'), ('##지만', 'O'), ('가가', 'O